In [ ]:
from collections import defaultdict
import math
import sys
sys.setrecursionlimit(1000000000)
graph = defaultdict(list)
visited = set()

cache = {}

def combination(n, k, mod):
    if k > n or k < 0:
        return 0
    k = min(k, n - k)

    if (n, k) in cache:
        return cache[(n, k)] % mod
    if k == 0 or k == n:
        cache[(n, k)] = 1
        return 1 % mod
    numerator = 1
    denominator = 1
    for i in range(1, k + 1):
        num = n - k + i
        den = i
        g = math.gcd(num, den)
        num //= g
        den //= g
        numerator = numerator * num % mod
        denominator = denominator * den % mod
    try:
        inverse_denominator = pow(denominator, -1, mod)
        result = numerator * inverse_denominator % mod
    except ValueError:
        result = 1
        for i in range(1, k + 1):
            result = result * (n - k + i) // i
        result %= mod

    cache[(n, k)] = result
    return result

def mod_factorial(n, mod):
    if n in cache and isinstance(cache[n], int):  
        return cache[n]
    result = 1
    for i in range(2, n + 1):
        result = (result * i) % mod
    cache[n] = result
    return result

def permutations(n, k, mod):
    result = 0
    for i in range(1, k + 1):
        result += combination(k - 1, i - 1, mod) * combination(n, i, mod)
        result %= mod
    return (mod_factorial(k, mod) * result) % mod

def full_sum(mod):
    full_sum_result = 1
    count_of_tries = 0
    vertexes = list(graph.keys())
    for vertex in vertexes:
        if vertex not in visited:
            if valid_first_neigh(vertex):
                temp_sum = one_sum(vertex, mod)
                if temp_sum == 0:
                    return 0
                if hollow_ones(1 if len(graph[vertex]) > 1 else 0, set(), vertex):
                    temp_sum *= 2
                else:
                    temp_sum *= 4
                full_sum_result = (full_sum_result * temp_sum) % mod
                count_of_tries += 1
    return (full_sum_result * mod_factorial(count_of_tries, mod)) % mod

def one_sum(vertex, mod):
    sum_for_current_tree = 1
    stack = [vertex]
    while stack:
        cur_vertex = stack.pop()
        if cur_vertex not in visited:
            visited.add(cur_vertex)
            neighbours = list(graph[cur_vertex])
            full_lonely_neighbours_sum = full_lonely_neighb(neighbours)
            neighbours = [x for x in neighbours if x not in visited]
            if len(neighbours) > 1:
                return 0
            elif len(neighbours) == 1:
                stack.append(neighbours[0])

                
            sum_for_current_tree = (sum_for_current_tree * mod_factorial(full_lonely_neighbours_sum, mod)) % mod
    return sum_for_current_tree

def full_lonely_neighb(neighbours):
    lonely = 0
    for neighbour in neighbours:
        if len(graph[neighbour]) == 1:
            lonely += 1
            visited.add(neighbour)
    return lonely

def hollow_ones(depth, temp_visited, vertex):
    stack = [(depth, vertex)]
    while stack:
        depth, vertex = stack.pop()
        if depth == 3:
            return False
        temp_visited.add(vertex)
        is_hollow = True
        for neighbour in graph[vertex]:
            if neighbour not in temp_visited:
                stack.append((depth + 1, neighbour))
        if not is_hollow:
            return False
    return True

def valid_first_neigh(vertex):
    count_bad_neighbours = 0
    neighbours = graph[vertex]
    bad_neighbour = -1
    for neighbour in neighbours:
        if len(graph[neighbour]) > 1:
            if bad_neighbour != -1:
                return False
            bad_neighbour = neighbour


    if bad_neighbour != -1:
        neighbours = graph[bad_neighbour]
        is_bad = False
        for neighbour in neighbours:
            if neighbour == vertex:
                continue
            if len(graph[neighbour]) > 1:
                if is_bad:
                    return False
                is_bad = True
        
    return True


def dfs_cycle_check(vertex, visited, parent):
    stack = [(vertex, parent)]
    while stack:
        cur_vertex, parent = stack.pop()
        visited.add(cur_vertex)
        for neighbour in graph[cur_vertex]:
            if neighbour not in visited:
                stack.append((neighbour, cur_vertex))
            elif neighbour != parent:
                return True
    return False


def is_tree():
    visited_local = set()
    for vertex in graph:
        if vertex not in visited_local:
            if dfs_cycle_check(vertex, visited_local, None):
                return False
    return len(visited_local) == len(graph)


def main():
    file = open('input.txt')
    n, k, mod = map(int, file.readline().split())

    for _ in range(k):
        a, b = map(int, file.readline().split())
        graph[a].append(b)
        graph[b].append(a)
        
    if not is_tree():
        print(0)
        return
    
    e = n - len(graph)
    if e == n:
        print(mod_factorial(e, mod) % mod)
    else:
        print((full_sum(mod) * (1 if e == 0 else permutations(n - e + 2, e, mod))) % mod)

if __name__ == "__main__":
    main()
